In [1]:
#Run the optimal kmeans to remove the benign cluster from data (without capture 9)
import pandas as pd
import os
from pycaret.clustering import *

In [2]:
#load data
os.chdir('../..') #to change working directory of the notebook when restarted
#Initialize empty df
df_all=pd.DataFrame()
for root, dirs, files in os.walk("./data/csv/captures_features"):
    for ctuName in files:
        #ignore capture 9
        if ctuName == 'capture20110817.binetflow.labels-positive-weights.labeled.csv':
            continue
        df=pd.read_csv("./data/csv/captures_features/"+ctuName)
        #add to general df as new rows
        df_all=df_all.append(df, ignore_index=True)
print(df_all.shape)


(2874213, 9)


In [3]:
#Change background and normal to 0 and infected values to 1
df_all['label']=df_all['label'].replace(['background','normal'],0)
df_all['label']=df_all['label'].replace(['infected'],1)
#drop node column
df_all=df_all.drop(['node'], axis=1)


df_all

,ID,OD,IDW,ODW,BC,LCC,AC,label
0,0,1,0,1639,0.000000e+00,0.0,1.000000,0
1,6,0,2113,0,0.000000e+00,0.0,16.895299,0
2,1,1,2,2,0.000000e+00,0.0,0.997643,0
3,135476,140755,1397778,1398024,3.080584e+10,0.0,-0.117842,0
4,1,1,7,7,0.000000e+00,0.0,0.999671,0
...,...,...,...,...,...,...,...,...
2874208,0,1,0,3,0.000000e+00,0.0,1.000000,0
2874209,0,1,0,6,0.000000e+00,0.0,1.000000,0
2874210,3,0,24,0,0.000000e+00,0.0,1.240000,0
2874211,0,1,0,6,0.000000e+00,0.0,1.000000,0


In [4]:
df_nolabel=df_all.drop(['label'], axis=1)
cluster = setup(df_nolabel, session_id = 7652,n_jobs=1)

,Description,Value
0,session_id,7652
1,Original Data,"(2874213, 7)"
2,Missing Values,False
3,Numeric Features,7
4,Categorical Features,0
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(2874213, 7)"
9,CPU Jobs,1


In [5]:
model = create_model('kmeans',num_clusters=25,n_init=1)

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,15:38:38
Status,. . . . . . . . . . . . . . . . . .,Fitting 25 Clusters
Estimator,. . . . . . . . . . . . . . . . . .,K-Means Clustering


,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness


In [6]:
kmeans_df = assign_model(model)
#add original labels to the df 
kmeans_df['label']=df_all['label']
kmeans_df

,ID,OD,IDW,ODW,BC,LCC,AC,Cluster,label
0,0,1,0,1639,0.000000e+00,0.0,1.000000,Cluster 0,0
1,6,0,2113,0,0.000000e+00,0.0,16.895299,Cluster 0,0
2,1,1,2,2,0.000000e+00,0.0,0.997643,Cluster 0,0
3,135476,140755,1397778,1398024,3.080584e+10,0.0,-0.117842,Cluster 14,0
4,1,1,7,7,0.000000e+00,0.0,0.999671,Cluster 0,0
...,...,...,...,...,...,...,...,...,...
2874208,0,1,0,3,0.000000e+00,0.0,1.000000,Cluster 0,0
2874209,0,1,0,6,0.000000e+00,0.0,1.000000,Cluster 0,0
2874210,3,0,24,0,0.000000e+00,0.0,1.240000,Cluster 0,0
2874211,0,1,0,6,0.000000e+00,0.0,1.000000,Cluster 0,0


In [ ]:
#get cluster centroids
#calculate the mean for each column for each cluster
Centroids_df=pd.DataFrame(kmeans_df.groupby('Cluster')['ID'].mean())
Centroids_df['OD']=kmeans_df.groupby('Cluster')['OD'].mean()
Centroids_df['IDW']=kmeans_df.groupby('Cluster')['IDW'].mean()
Centroids_df['ODW']=kmeans_df.groupby('Cluster')['ODW'].mean()
Centroids_df['BC']=kmeans_df.groupby('Cluster')['BC'].mean()
Centroids_df['LCC']=kmeans_df.groupby('Cluster')['LCC'].mean()
Centroids_df['AC']=kmeans_df.groupby('Cluster')['AC'].mean()
Centroids_df

In [ ]:
#drop data inside the benign cluster
#drop most common cluster
benign_str=kmeans_df['Cluster'].value_counts().idxmax()
phase2_data=kmeans_df[kmeans_df['Cluster']!=benign_str]
#drop Cluster column
phase2_data.drop(['Cluster'], axis=1, inplace=True)
phase2_data


In [ ]:
#check hob and bob te verify right run
phase2_data['label'].value_counts()

In [ ]:
#save data to csv
phase2_data.to_csv('./data/csv/phase2_data.csv', index=False)
Centroids_df.to_csv('./data/csv/phase1_centroids.csv', index=False)